# Homework 4 - Mingxuan Wang

Github Username: Michelle-MingxuanWang  
Github Repository: https://github.com/Michelle-MingxuanWang/Michelle-MingxuanWang.github.io  
Blog: https://michelle-mingxuanwang.github.io

In [74]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output

# Read and tranform the dataset

In [75]:
df = pd.read_excel('C:\\1_Work\\Duke_3rd_semester\\823\\hm\\hw4\\sed17-sr-tab006.xlsx', skiprows=3, header=[0,1])
df=df.drop(df.index[0])
df.insert(0,'State code',["AL","AK","AZ","AR","CA","CO","CT","DE","DC","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","PR","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"])
df = df.set_index([('State or location', 'Unnamed: 0_level_1'),'State code'])\
    .stack().reset_index()\
    .set_index([('State or location', 'Unnamed: 0_level_1'),'State code','level_2'])\
    .stack().reset_index()
df.columns = ['States','State code','Sex','Field','Doctorates Awarded']
df = df[df["Field"]!="Totala"]
df["Doctorates Awarded"].replace("D",np.NaN,inplace=True)
df

,States,State code,Sex,Field,Doctorates Awarded
0,Alabama,AL,Female,Education,81.0
1,Alabama,AL,Female,Engineering,31.0
2,Alabama,AL,Female,Humanities and arts,15.0
3,Alabama,AL,Female,Life sciencesb,92.0
4,Alabama,AL,Female,Mathematics and computer sciences,13.0
...,...,...,...,...,...
930,Wyoming,WY,Male,Life sciencesb,19.0
931,Wyoming,WY,Male,Mathematics and computer sciences,NaN
932,Wyoming,WY,Male,Otherc,NaN
933,Wyoming,WY,Male,Physical sciences and earth sciences,11.0


# Create a dataset to use in choropleth map in Dash

In [76]:
df_state = df.groupby("State code",as_index=False)["Doctorates Awarded"].sum()

# Analysis in Pandas

In [77]:
df.groupby("States").sum().sort_values(by="Doctorates Awarded",ascending=False).head()

,Doctorates Awarded
States,
California,6100.0
Texas,4065.0
New York,4064.0
Massachusetts,2878.0
Pennsylvania,2626.0


The top 5 states with the highest doctorates awarded are California, Texas, New York, Massachusetts, and Pennsylvania.

In [78]:
df.groupby("Field").sum().sort_values(by="Doctorates Awarded",ascending=False).head()

,Doctorates Awarded
Field,
Life sciencesb,12554.0
Engineering,9693.0
Psychology and social sciences,9027.0
Physical sciences and earth sciences,6005.0
Humanities and arts,5222.0


The top 5 fields with the highest doctorates awarded are "Life Science", "Engineering", "Psychology and social sciences", "Physical sciences and earth sciences", and "Humanities and arts".

In [79]:
df.groupby("Sex").sum().sort_values(by="Doctorates Awarded",ascending=False) / df["Doctorates Awarded"].sum()

,Doctorates Awarded
Sex,
Male,0.532396
Female,0.467604


Among all the doctorates awarded, 53.24% are male while 46.76% are female.

In [80]:
(df[df["Sex"]=="Female"].groupby("Field").sum() / df[df["Sex"]=="Male"].groupby("Field").sum()).sort_values(by="Doctorates Awarded",ascending=False)

,Doctorates Awarded
Field,
Education,2.156731
Psychology and social sciences,1.450991
Otherc,1.244577
Life sciencesb,1.235001
Humanities and arts,1.050255
Physical sciences and earth sciences,0.494525
Mathematics and computer sciences,0.344510
Engineering,0.333654


Among all the field, Education has the highest female to male ratio, Engineering has the lowest female to male ratio.   
In fields of "Education", "Psychology and social sciences", "Others", "Life sciences", "Humanities and arts", the number of female doctorates awarded outnumbers the number of male doctorates awarded; while in fields of "Physical sciences and earth sciences", "Mathematics and computer sciences", and "Engineering", the number of male doctorates awarded outnumbers the number of female doctorates awarded.

# Create a dashboard in Dash

In [ ]:
app = dash.Dash()

app.layout = html.Div([
    html.Div([html.H1("Doctorates Recipients by State, Field of Study, and Sex", style={'text-align': 'center'})]),
    html.Div([
        html.Div([html.H2("Doctorates Awarded by State"),
                  html.H3("Hover on State for Detailed Plots"),
                  dcc.Graph(id='map', figure=px.choropleth(
                  data_frame=df_state,
                    locationmode='USA-states',
                    locations='State code',
                    scope="usa",
                    color='Doctorates Awarded',
                    hover_data=['State code', 'Doctorates Awarded'],
                    color_continuous_scale="deep",
                  custom_data=['State code']#,
                  #title='Doctorates Awarded by State'
              ))
                 ],style={'width': '49%', 'float': 'left','display': 'inline-block'}),
        html.Div([
            html.H2("Doctorates Awarded by Sex", style={'text-align': 'left'}),
            dcc.Dropdown(id="slct_field",
                         options=[{'label': i, 'value': i} for i in df['Field'].unique()],
                         multi=False,
                         value="Education",
                         style={'width': "50%"}),
            dcc.Graph(id='sex_plot',figure={})
        ], style={'display': 'inline-block', 'float': 'right', 'width': '49%'})
    ]),
    html.Div([
            html.H2("Doctorates Awarded by Field", style={'text-align': 'left'}),
            dcc.Dropdown(id="slct_sex",
                         options=[{'label': i, 'value': i} for i in df['Sex'].unique()],
                         multi=False,
                         value="Male",
                         style={'width': "50%"}),
            dcc.Graph(id='field_plot',figure={})
    ])
])

@app.callback(
    dash.dependencies.Output(component_id='field_plot', component_property='figure'),
    [dash.dependencies.Input(component_id='map', component_property='hoverData'),\
     dash.dependencies.Input(component_id='slct_sex', component_property='value')]
)

def update_graph(hoverData,option_slct_sex):
    if hoverData is None:
        df_slct_sex = df[df["Sex"] == option_slct_sex]
        fig_field = px.bar(df_slct_sex, x='Field', y='Doctorates Awarded',color="Field",\
                           color_discrete_sequence=px.colors.qualitative.Pastel,title=f'Doctorates Awarded')
        return fig_field
    else:
        state_code = hoverData['points'][0]['customdata'][0]
        df_slct_sex = df[df['State code'] == state_code]
        df_slct_sex = df_slct_sex[df_slct_sex["Sex"] == option_slct_sex]

        fig_field = px.bar(df_slct_sex, x='Field', y='Doctorates Awarded',color="Field",\
                           color_discrete_sequence=px.colors.qualitative.Pastel,title=f'{option_slct_sex} Doctorates Awarded in {state_code}')

        return fig_field

@app.callback(
    dash.dependencies.Output(component_id='sex_plot', component_property='figure'),
    [dash.dependencies.Input(component_id='map', component_property='hoverData'),\
     dash.dependencies.Input(component_id='slct_field', component_property='value')
     ]
)
def update_graph(hoverData,option_slct_field):
    if hoverData is None:
        df_slct_field = df[df["Field"] == option_slct_field]
        fig_sex = px.pie(df_slct_field, values='Doctorates Awarded', names='Sex',color="Sex",\
                           color_discrete_sequence=px.colors.qualitative.Pastel,title=f'Doctorates Awarded')
        return fig_sex
    else:
        state_code = hoverData['points'][0]['customdata'][0]
        df_slct_field = df[df['State code'] == state_code]
        df_slct_field = df_slct_field[df_slct_field["Field"] == option_slct_field]

        fig_sex = px.pie(df_slct_field, values='Doctorates Awarded', names='Sex',color="Sex",\
                           color_discrete_sequence=px.colors.qualitative.Pastel,title=f'Doctorates Awarded in {state_code} in Field of {option_slct_field}')
        return fig_sex

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Oct/2021 21:33:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:33:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:33:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:33:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:33:44] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:33:44] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:33:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:33:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:34:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:34:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 21:34:03] "POST /_dash-update-component HTTP/1.